In [ ]:
#!/usr/bin/env python3
"""
James Gardner 2019

"It is important to note that OpenCV reads colors as BGR (Blue Green Red),
where most computer applications read as RGB (Red Green Blue).
Remember this!"
"""

import cv2
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm

In [ ]:
def zoom(frame,r=0.5):
    rheight = int(frame.shape[0]*r)
    rwidth  = int(frame.shape[1]*r)
    # caution: order of width, height in resize call
    return cv2.resize(frame,(rwidth,rheight))

In [ ]:
cap = cv2.VideoCapture('tracking_test.mp4')
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
frame_count = 0
pbar = tqdm(total=total_frames)

ret, frame = cap.read()

while ret:
    cv2.imshow('a',zoom(frame[:,:,1]))

    # show frame for 30 ms 
    # 0xFF necessary on 64-bit to truncate waitKey, & is bit-wise
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break    
    
    ret, frame = cap.read()
    frame_count += 1
    pbar.update(1)

cv2.destroyAllWindows()
pbar.close()
cap.release()

In [ ]:
# openCV image tracking tutorial
# https://opencv-python-tutroals.readthedocs.io/en/latest/
# py_tutorials/py_video/py_lucas_kanade/py_lucas_kanade.html
# https://stackoverqflow.com/questions/43063320/cv2-calcopticalflowpyrlk-adding-new-points

cap = cv2.VideoCapture('tracking_test.mp4')
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
frame_count = 0
pbar = tqdm(total=total_frames)

# params for ShiTomasi corner detection,
feature_params = dict( maxCorners = 100,
                       qualityLevel = 0.3,
                       minDistance = 7,
                       blockSize = 7 )

# Parameters for lucas kanade optical flow
lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# Create some random colors
color = np.random.randint(0,255,(100,3))

# Take first frame and find corners in it
ret, old_frame = cap.read()
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
# p0 = cv2.goodFeaturesToTrack(old_gray, mask = None, **feature_params)
fav_p = []
p0 = np.array([[1093,548],[1226,544],[1344,547],[1439,553]],np.float32)
# print(p0)
# 427 175 , 401 187
# p0 = np.array([[[427,175]],[[401,187]]])

# Create a mask image for drawing purposes
mask = np.zeros_like(old_frame)
ret,frame = cap.read()

after_first = False

while ret:
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
    # calculate optical flow
    p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)

    # Select good points
    #print(p1,p0,st,err)
    good_new = p1#[st==1]
    good_old = p0#[st==1]

    # draw the tracks
    for i,(new,old) in enumerate(zip(good_new,good_old)):
        a,b = new.ravel()
        c,d = old.ravel()
        mask = cv2.line(mask, (a,b),(c,d), color[i].tolist(), 2)
        frame = cv2.circle(frame,(a,b),5,color[i].tolist(),-1)
    img = cv2.add(frame,mask)

    #cv2.imshow('frame',zoom(img))
    #if cv2.waitKey(30) & 0xff == ord('q'):
    #    break

    # Now update the previous frame and previous points
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1,1,2)
    frame_count += 1
    pbar.update(1)
    
    if after_first:
        fav_p.append([p[0][0] for p in p1])    
    else:
        after_first = True
    
    ret,frame = cap.read()

fav_p = np.array(fav_p)
pbar.close()    
cv2.destroyAllWindows()
cap.release()

In [ ]:
plt.figure(figsize=(7,14))
plt.imshow(old_frame)#, cmap = 'gray', interpolation = 'bicubic')
# plt.xticks([]), plt.yticks([])  # to hide tick values on X and Y axis
plt.plot(1200,600,'w.')
plt.show()


In [ ]:
p1,p1.shape

In [ ]:
lyst = []
for p in p1:
    lyst.append(p[0][0])

In [ ]:
# img = cv2.imread()
cap = cv2.VideoCapture('tracking_test.mp4')
ret, frame = cap.read()
g_frame = frame[:,:,1]
# print(g_frame.shape)

# cv2.namedWindow('a',cv2.WINDOW_NORMAL)
# cv2.namedWindow('a',cv2.WINDOW_AUTOSIZE) 
cv2.imshow('a',zoom(g_frame))
cv2.waitKey(0)
cv2.destroyAllWindows()
cap.release()

In [ ]:
cap = cv2.VideoCapture('tracking_test.mp4')
# ret is 'did it successfully return another frame?'

while True:
    if cv2.waitKey(1) == ord('q'):
        break
    
    ret, frame = cap.read()
    
    try:
        grey = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        cv2.imshow('frame',frame)
        cv2.imshow('greyframe',grey)
    except:
        cap.release()
        cv2.destroyAllWindows()        
        break    

cap.release()
cv2.destroyAllWindows()

In [ ]:
img = cv2.imread('bgr_tester.jpeg')

b_channel = img[:,:,0]
g_channel = img[:,:,1]
r_channel = img[:,:,2]

cv2.imshow('b',b_channel)
cv2.imshow('g',g_channel)
cv2.imshow('r',r_channel)
# waitKey argument is the number of milliseconds it waits for
if cv2.waitKey(0) == ord('q'):
    # cv2.waitKey(0)
    cv2.destroyAllWindows()

In [ ]:
# frame splitter!
cap = cv2.VideoCapture('tracking_test.mp4')
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
frame_count = 0
pbar = tqdm(total=total_frames)

ret = True
while ret:    
    ret, frame = cap.read()
    if not ret:
        break
        
    #cv2.imwrite('./plots/tracking_test_{:03d}.jpeg'.format(frame_count),frame)
    
    frame_count += 1
    pbar.update(1)
    
pbar.close()
cap.release()
cv2.destroyAllWindows()

In [ ]:
# img = cv2.imread('squid.jpg')#,cv2.IMREAD_GRAYSCALE)
# print(img.shape)
# face = img[50:250,150:350]
# img[400:600,200:400] = face

# plt.figure(figsize=(7,14))
# plt.imshow(img)#, cmap = 'gray', interpolation = 'bicubic')
# # plt.xticks([]), plt.yticks([])  # to hide tick values on X and Y axis
# plt.plot([200,300,400],[100,200,300],'c', linewidth=5)
# plt.show()

# # cv2.imwrite('watchgray.png',img)